In [2]:
import numpy as np
import pandas as pd
import glob
import sys,os
from pathlib import Path

In [3]:
from PIL import Image
import re
from tqdm import tqdm

### 0.API

In [ ]:
sys.path.append('../../')
from utils.plots.plots import draw_bbox

In [ ]:
# id_standard_df.to_csv('/home/chenyi/workspace/myPyModule99/dataset/labelMap/standard/id2class99_chenyi.csv', index=False)

In [ ]:
id_standard_df = pd.read_csv('/home/chenyi/workspace/myPyModule99/dataset/labelMap/standard/id2class99_chenyi.csv')
id_use_df = id_standard_df[id_standard_df['use']>=0]

In [ ]:
with open('/home/chenyi/workspace/myPyModule99/dataset/labelMap/standard/haowei34k_map_stand.json', 'r') as f:
    hw2stand = json.load(f)

In [5]:
def get_list2dict(k_list, v_list):
    d = {}
    for item in zip(k_list, v_list):
        d[item[0]] = item[1]
    return d

In [6]:
model_label_map = {
    0: 10,  # 鞋子
    1: 8,  # 包包
    2: 4,  # 上装
    3: 5,  # 裤子
    4: 6,  # 裙子
    5: 7,  # 连体装
}

In [ ]:
# l2_map_l1 = get_list2dict( id_use_df['label99'].values, id_use_df['label_l1'].values)

In [7]:
label2model = get_list2dict( model_label_map.values(), model_label_map.keys())

### 1.Dataset Information.

In [7]:
root_nas = Path('/nas/chenyi/workspace/myPyModule/dataset/data')
!tree -d -L 1 $root_data

/nas/chenyi/workspace/myPyModule/dataset/data
├── bboxes
├── dict
├── @eaDir
├── slink
└── watoo

5 directories


In [5]:
root_yolo = Path('/home/chenyi/workspace/yolov5/datasets/')
!tree -d -L 1 $root_yolo

/home/chenyi/workspace/yolov5/datasets
├── coco128
├── dadet100kv2
├── dadet100kv3_june
├── dadet100kv4_1_june
├── dadet100kv4_june
├── dadet_gallv4_june
├── dadetv5
├── dadetv5_query
├── dataExps
├── deepedia
├── deepedia_catg6
├── deepedia_catg8
├── haowei_bag_shoe
├── hwpedia
├── hwpedia70k
├── hwpedia80k
├── tuneGallery10k
├── tuneGallery5k
├── tuneQuery10k
└── validation_all

20 directories


In [6]:
!ls $root_data/

bboxes	dict  @eaDir  slink  watoo


In [7]:
!ls $root_data/bboxes

dadet100k     @eaDir  fashionpedia  hwpedia  tuneDataset
deepfashion2  exps    haowei34k     query


In [499]:
!ls $root_data/bboxes/dadet100k

dadet100kv4_1_bboxes.csv  dadet_queryv5_bboxes.csv  dadetv5_slink.csv
dadet100kv5_bboxes.csv	  dadetv5_query_slink.csv   @eaDir


In [29]:
# !mkdir $root_data/data/bboxes

##### 读取csv

In [5]:
root_nas = Path('/nas/chenyi/datasets_nas/dada_det/xml/')
root_xml = root_nas /'wardrobe500k/wardrobe20k_xml_hard20220705'
!ls $root_xml

info_csv  xml


In [8]:
bboxes_df = pd.read_csv(root_xml /'info_csv/xml_clr_20220705.csv')
len(bboxes_df)

20971

In [9]:
bboxes_df.head(1)

,xml_path,width,height,labelimg,bx1,bx2,bx3,bx4,stem,img_sp,label86
0,/nas/chenyi/datasets_nas/wardrobe20k_labelimg/...,1080,1554,裙子,6,478,1080,1129,2017022320122485968,/nas/chenyi/datasets_nas/wardrobe20k_labelimg/...,4


##### 数据筛选

In [14]:
bboxes_df.describe()

,label_pred,bx1,bx2,bx3,bx4,bbox_ind,width,height
count,312242.000000,312242.000000,312242.000000,312242.000000,312242.000000,312242.000000,312242.000000,312242.000000
mean,1.637682,0.494991,0.644653,0.279343,0.296352,1.400084,745.858578,1017.250607
std,1.673030,0.099545,0.217277,0.212213,0.216663,1.328984,442.344862,524.914307
min,0.000000,0.021207,0.016602,0.016667,0.011418,0.000000,30.000000,40.000000
25%,0.000000,0.448806,0.455566,0.106421,0.110000,0.000000,550.000000,750.000000
50%,1.000000,0.497653,0.628906,0.234211,0.253906,1.000000,682.000000,1024.000000
75%,3.000000,0.540996,0.878418,0.376440,0.416667,2.000000,767.000000,1024.000000
max,5.000000,0.979492,0.989852,0.999752,0.999752,25.000000,6000.000000,6286.000000


In [504]:
bboxes_df.head(1)

,label_pred,bx1,bx2,bx3,bx4,bbox_ind,label_sp,stem,label_tp,mode,img_sp,source,img_tp
0,0,0.44003,0.904297,0.13943,0.060547,0,/home/chenyi/workspace/dataset/labels/labels6/...,551df69f15ca760f19dd6f40bfda2aa7,/home/chenyi/workspace/yolov5/datasets/dadet10...,train,/home/chenyi/workspace/dataset/data/data_fashi...,data_fashionpedia,/home/chenyi/workspace/yolov5/datasets/dadet10...


In [15]:
bboxes_df.value_counts('source')

source
haowei34k            143966
data_fashionpedia    125609
dada_det              26893
DeepFashion2           9179
shoeBag5k0515          6595
dtype: int64

In [506]:
bboxes_g_df = bboxes_df[(bboxes_df['source']=='haowei34k') | (bboxes_df['source']=='data_fashionpedia')]

In [507]:
bboxes_q_df = bboxes_df[~((bboxes_df['source']=='haowei34k') | (bboxes_df['source']=='data_fashionpedia'))]

In [509]:
len(bboxes_g_df), len(bboxes_q_df)

(269575, 42667)

In [510]:
bboxes_df.value_counts('label_pred').sort_index()

label_pred
0    121509
1     35677
2     71233
3     34183
4     17539
5     32101
dtype: int64

In [511]:
len(bboxes_df)

312242

In [512]:
bboxes_g_df = bboxes_g_df[bboxes_g_df['label_pred']>=2]

In [513]:
bboxes_df = pd.concat([bboxes_g_df, bboxes_q_df], axis=0)

In [16]:
len(bboxes_df)

312242

### 画图

In [ ]:
df = openfashion_pd

In [ ]:
df.head(2)

In [ ]:
gp = dict(list(df.groupby('img_sp')))

In [ ]:
gp[list(gp.keys())[0]]

In [ ]:
import cv2
TROOT = '/nas/chenyi/datasets_cls/haowei34k_drawed/images/'
for k,v in tqdm(gp.items()):
    k_p = Path(k)
    img_np = cv2.imread(k)
    # img_np = cv2.cvtColor(img_np, cv2.COLOR_BGR2RGB)
    bboxes = v.loc[:, ['label2_eng', 'xmin', 'ymin', 'xmax', 'ymax']].values
    mode = v['mode'].values[0]
    tp = Path('{0}{1}/{2}/{3}'.format(TROOT, mode, np.random.randint(0,1000), k_p.name))
    for bx in bboxes:
        img_np = draw_bbox(img_np, bx, fontScale=1)
    if not tp.parent.is_dir():
        tp.parent.mkdir(exist_ok=True, parents=True)
    try:
        cv2.imwrite(str(tp),img_np)
    except:
        print('error: p')
    # break

In [ ]:
slink_df = pd.read_csv('./data/hw34k_level1_slink.csv')

In [ ]:
slink_df.head(2)

In [ ]:
fname2path = get_list2dict(slink_df['fname'].values, slink_df['image_spath'].values)

In [ ]:
openfashion_pd['img_sp'] = openfashion_pd['fname'].map(fname2path)

In [ ]:
openfashion_pd.head()

In [ ]:
counter = openfashion_pd.value_counts('label8').sort_index()

In [ ]:
id_list = np.array(counter.axes)[0]
numbers = counter.values
name_list = [str(x) + ':' + id2eng_l1[x] for x in id_list]

In [ ]:
plt.figure(figsize=(25, 7))
plt.xticks(list(range(len(name_list))), name_list, rotation=75, fontsize=14)
plt.bar(name_list, numbers, width=0.5)  # 绘制y刻度标签
for a, b in zip(name_list, numbers):
    plt.text(a, b, b, ha='center', va='bottom')
plt.show()

### 解析label.txt

In [9]:
def read_txt(p):
    with open(str(p), 'r') as f:
        label_lines = f.readlines()
    return label_lines

In [10]:
def get_bboxes_info(p):
    import numpy as np
    lines_list = []
    for ind,line in enumerate(read_txt(p)):
        line_list = line.split('\n')[0].split(' ')
        line_list.append(ind)
        line_list.append(p)
        lines_list.append(line_list)
    return lines_list

In [11]:
global bboxes_all
def get_bboxes_multithread(p):
    bboxes_info = get_bboxes_info(p)
    bboxes_all += bboxes_info

In [23]:
!tree -d -L 1 /home/chenyi/workspace/yolov5/datasets/dadet100kv4_june

/home/chenyi/workspace/yolov5/datasets/
├── coco128
├── dadet100kv2
├── dadet100kv3_june
├── dadet100kv4_1_june
├── dadet100kv4_june
├── dadet_gallv4_june
├── dataExps
├── deepedia
├── deepedia_catg6
├── deepedia_catg8
├── haowei_bag_shoe
├── hwpedia
├── hwpedia70k
├── hwpedia80k
├── tuneGallery10k
├── tuneGallery5k
├── tuneQuery10k
└── validation_all

18 directories


In [41]:
folder_name = 'dadet100kv4_1_june'

In [48]:
root_dataset = Path(f'/home/chenyi/workspace/yolov5/datasets/{folder_name}/labels/*/*.txt')

In [44]:
labels_p = glob.glob(str(root_dataset))
len(labels_p)

103123

In [114]:
labels_p = glob.glob('/home/chenyi/workspace/dataset/labels/dada_det/labels202205/shoeBag5k0515/*/*')

In [115]:
from tqdm import tqdm
bboxes_all = []
for p in tqdm(labels_p):
    bboxes_info = get_bboxes_info(p)
    bboxes_all += bboxes_info
    # break

100%|██████████████████████████████████████| 4430/4430 [00:00<00:00, 8784.26it/s]


In [ ]:
# %%time
# # global bboxes_all
# # bboxes_all = []
# run_task_multithread(labels_df['crops_p'][:10], get_bboxes_multithread, 20)

#### 处理bbox信息

In [53]:
root_nas = Path('/nas/chenyi/workspace/myPyModule/dataset/data')

In [54]:
!ls $root_data/bboxes

dadet100k     @eaDir  fashionpedia  hwpedia  tuneDataset
deepfashion2  exps    haowei34k     query


In [57]:
!ls $root_data/bboxes/dadet100k

bboxes_all_df.csv


In [142]:
# bboxes_df = pd.DataFrame(bboxes_all)

In [344]:
bboxes_df = pd.read_csv(root_nas /'bboxes/dadet100k/dadet100kv5_bboxes.csv')
# df.to_csv(root_nas /'bboxes/dadet100k/dadet100kv5_bboxes.csv', index=False)

In [345]:
len(bboxes_df)

312242

In [346]:
bboxes_df.head(1)

,label_pred,bx1,bx2,bx3,bx4,bbox_ind,label_sp,stem,label_tp,mode,img_sp,source,img_tp
0,0,0.44003,0.904297,0.13943,0.060547,0,/home/chenyi/workspace/dataset/labels/labels6/...,551df69f15ca760f19dd6f40bfda2aa7,/home/chenyi/workspace/yolov5/datasets/dadet10...,train,/home/chenyi/workspace/dataset/data/data_fashi...,data_fashionpedia,/home/chenyi/workspace/yolov5/datasets/dadet10...


In [144]:
# name_stand = ['label_pred', 'bx1', 'bx2', 'bx3', 'bx4', 'bbox_ind', 'label_sp']
# bboxes_df.rename(columns={bboxes_df.columns[0]:name_stand[0],
#                           bboxes_df.columns[1]:name_stand[1],
#                           bboxes_df.columns[2]:name_stand[2],
#                           bboxes_df.columns[3]:name_stand[3],
#                           bboxes_df.columns[4]:name_stand[4],
#                           bboxes_df.columns[5]:name_stand[5],
#                           bboxes_df.columns[6]:name_stand[6],
#                          },
#                           inplace=True)

In [186]:
len(bboxes_df)

318837

In [173]:
# bboxes_df['label_tp'] = bboxes_df['label_sp'].apply(lambda x: '/home/chenyi/workspace/yolov5/datasets/dadet100kv4_1_june/labels/{0}/{1}'.format(Path(x).parts[-2], Path(x).name))

In [175]:
# bboxes_df['img_tp'] = bboxes_df['img_sp'].apply(lambda x: '/home/chenyi/workspace/yolov5/datasets/dadet100kv4_1_june/images/{0}/{1}'.format(Path(x).parts[-2], Path(x).name))

In [145]:
bboxes_df['stem'] = bboxes_df['label_tp'].apply(lambda x: Path(x).stem)

In [153]:
bboxes_df['mode'] = bboxes_df['label_tp'].apply(lambda x: Path(x).parts[-2])

In [159]:
bboxes_df['img_sp'] = bboxes_df['label_tp'].apply(lambda x: f'/home/chenyi/workspace/dataset/data/dada_det/shoeBag5k0515/images/{x}')
bboxes_df['img_sp'] = bboxes_df['img_sp'].apply(lambda x: x.replace('.txt', '.jpg'))

In [154]:
# bboxes_df['label_sp'] = bboxes_df['label_tp'].apply(lambda x: os.readlink(x))

In [164]:
bboxes_df['source'] = bboxes_df['img_sp'].apply(lambda x: Path(x).parts[-4])

In [96]:
bboxes_df['source_l2'] = bboxes_df['img_sp'].apply(lambda x: Path(x).parts[7])

In [317]:
bboxes_df.value_counts('source')

source
haowei34k            143966
data_fashionpedia    125609
dada_det              26893
DeepFashion2           9179
shoeBag5k0515          6595
dtype: int64

In [126]:
bboxes_df.value_counts('source_l2')

source_l2
images               143966
train                131628
shoe_bag11k_may       14119
wholebody0601_all     12714
haowei23               6595
validation             3160
hard_sample_0610         60
dtype: int64

In [95]:
!tree -d -L 2 /home/chenyi/workspace/dataset/data/dada_det/shoeBag5k0515/images

/home/chenyi/workspace/dataset/data/dada_det
├── hard_sample_0610
│   └── @eaDir
├── __MACOSX
│   └── wholebody0601_all
├── shoe_bag11k_may
│   ├── bag_setdiff
│   ├── leisure_shoe_setdiff
│   └── suit_shoe_setdiff
├── shoeBag5k0515
│   └── images
└── wholebody0601_all
    ├── beidaiku2822_rename
    └── liantiku2500_0602_rename

13 directories


In [106]:
!tree -L 2 /home/chenyi/workspace/dataset/labels/dada_det

/home/chenyi/workspace/dataset/labels/dada_det
├── labels100k_skirt_df2
│   ├── train
│   └── validation
├── labels202205
│   └── shoeBag5k0515
├── labels202206
│   ├── labels_shoeBag11k
│   └── wholebody_june
└── labels_sys_error
    └── labels60_june

10 directories, 0 files


In [75]:
bboxes_df['img_tp'] = bboxes_df['label_tp'].apply(lambda x: x.replace('labels', 'images'))
bboxes_df['img_tp'] = bboxes_df['img_tp'].apply(lambda x: x.replace('.txt', '.jpg'))

In [77]:
bboxes_df['img_sp'] = bboxes_df['img_tp'].apply(lambda x: os.readlink(x))

### target label

In [26]:
!tree -d -L 2 /home/chenyi/workspace/dataset/labels/labels6/

/home/chenyi/workspace/dataset/labels/labels6/
├── fashionpedia
│   └── labels
├── haowei34k
│   └── labels
├── labels_exp
│   ├── dadetGallv4_del
│   └── delBbox
├── train
├── val
└── wardrobe500k
    └── labels10k_20220629

11 directories


In [29]:
!ls /nas/chenyi/datasets_nas/labels/

dada_det  labels6


In [31]:
root_labels = Path('/nas/chenyi/datasets_nas/labels/wardrobe500k/wardrobe20k_labels6')

In [30]:
# !mkdir -p /nas/chenyi/datasets_nas/labels/wardrobe500k/wardrobe20k_labels6

In [412]:
# df = pd.read_csv('./data/exps/dadetGallv4_del.csv')
df = bboxes_df

In [413]:
df.head(1)

,label_pred,bx1,bx2,bx3,bx4,bbox_ind,label_sp,stem,label_tp,mode,img_sp,source,img_tp
2,3,0.828336,0.753906,0.103448,0.101562,2,/home/chenyi/workspace/dataset/labels/labels6/...,551df69f15ca760f19dd6f40bfda2aa7,/home/chenyi/workspace/yolov5/datasets/dadet10...,train,/home/chenyi/workspace/dataset/data/data_fashi...,data_fashionpedia,/home/chenyi/workspace/yolov5/datasets/dadet10...


In [414]:
len(df)

306304

In [517]:
df.value_counts('label_pred').sort_index()

label_pred
0     59841
1     15162
2    107728
3     49284
4     22571
5     51718
dtype: int64

#### source label path

In [32]:
!ls $root_labels

In [534]:
root = '/home/chenyi/workspace/dataset/labels/query6/dadetv5_query/labels'

In [535]:
df['label_sp'] = df.apply(lambda row: Path('{0}/{1}/{2}.txt'.format(root_labels, row['mode'], row['stem'])), axis=1)

#### 生成txt

In [10]:
sys.path.append('../../')

In [11]:
from utils.bbox.conversation import bbox_coco2voc,bbox_voc2yolo,bbox_yolo2voc

In [521]:
# df['label_model'] = df['label_model'].map(label2model)

In [12]:
df = bboxes_df

In [14]:
df.head(1)

,xml_path,width,height,labelimg,bx1,bx2,bx3,bx4,stem,img_sp,label86
0,/nas/chenyi/datasets_nas/wardrobe20k_labelimg/...,1080,1554,裙子,6,478,1080,1129,2017022320122485968,/nas/chenyi/datasets_nas/wardrobe20k_labelimg/...,4


In [17]:
df['label_model'] = df['label86'].values

In [20]:
df.loc[df[df['label_model']==6].index, 'label_model'] = 2
df.loc[df[df['label_model']==7].index, 'label_model'] = 3

In [21]:
df.value_counts('label_model').sort_index()

label_model
0     574
1     751
2    8313
3    4904
4    4670
5    1759
dtype: int64

In [24]:
# yolo
bboxes_list = []
for ind,row in tqdm(df.iterrows()):
    bbox7 = row.loc[['label_model', 'bx1', 'bx2', 'bx3', 'bx4', 'width', 'height']].values.tolist()
    bbox5 = bbox7[:6]
    bbox5[1:] = bbox_voc2yolo(bbox5[1:], bbox7[5], bbox7[6])
    bbox5_str = '{0:.0f} {1:.6f} {2:.6f} {3:.6f} {4:.6f}\n'.format(*bbox5)
    bboxes_list.append(bbox5_str)
    # break

20971it [00:04, 5067.98it/s]


In [438]:
# voc2yolo
# bboxes_list = []
# for ind,row in tqdm(df.iterrows()):
#     bbox7 = row.loc[['label_pred', 'bx1', 'bx2', 'bx3', 'bx4', 'width', 'height']].values.tolist()
#     bbox5 = bbox7[:6]
#     # bbox5[1:] = bbox_voc2yolo(bbox5[1:], bbox7[5], bbox7[6])
#     bbox5_str = '{0:.0f} {1:.6f} {2:.6f} {3:.6f} {4:.6f}\n'.format(*bbox5)
#     bboxes_list.append(bbox5_str)
#     # break

In [23]:
bboxes_list[-1]

'4 0.502778 0.517053 0.994444 0.418919\n'

In [25]:
def write_txt(label_path, text):
    # label_path, text = input_list[0], input_list[1]
    label_path = Path(label_path)
    p_parent = label_path.parent
    if not p_parent.is_dir():
        p_parent.mkdir(parents=True)
    if not p_parent.is_file():
        with open(label_path, 'a+') as f:
            f.write(text)

In [538]:
Path(df['label_sp'].values[0]).parents[1]

PosixPath('/home/chenyi/workspace/dataset/labels/query6/dadetv5_query/labels')

In [ ]:
# import shutil
# if Path(df['label_sp'].values[0]).parents[1].is_dir():
#     shutil.rmtree(Path(df['label_sp'].values[0]).parents[1])

In [540]:
for label_path, text in tqdm(zip(df['label_sp'].values, bboxes_list)):
    write_txt(label_path, text)
    # break

180695it [00:04, 37924.59it/s]


In [541]:
label_path,text

(PosixPath('/home/chenyi/workspace/dataset/labels/query6/dadetv5_query/labels/val/2018041313545637741.txt'),
 '0 0.306250 0.525781 0.393750 0.620313\n')

In [542]:
!cat $label_path

0 0.712500 0.465625 0.325000 0.631250
0 0.306250 0.525781 0.393750 0.620313


#### 生成软连接的path

In [36]:
!tree -d -L 1 /home/chenyi/workspace/dataset/data/

/home/chenyi/workspace/dataset/data/
├── dada_det
├── data_fashionpedia
├── deepedia
├── DeepFashion2
├── haowei34k
└── tar

6 directories


In [37]:
!tree -d -L 1 /home/chenyi/workspace/yolov5/datasets/

/home/chenyi/workspace/yolov5/datasets/
├── coco128
├── dadet100kv2
├── dadet100kv3_june
├── dadet100kv4_1_june
├── dadet100kv4_june
├── dadet_gallv4_june
├── dataExps
├── deepedia
├── deepedia_catg6
├── deepedia_catg8
├── haowei_bag_shoe
├── hwpedia
├── hwpedia80k
├── tuneGallery10k
├── tuneGallery5k
├── tuneQuery10k
└── validation_all

17 directories


In [36]:
# !mkdir -p /home/chenyi/workspace/yolov5/datasets/dataExps/gall10k_delBox

In [543]:
df = bboxes_df

In [544]:
len(df)

180695

In [545]:
df.head(1).values

array([[3, 0.828336, 0.753906, 0.103448, 0.101562, 2,
        PosixPath('/home/chenyi/workspace/dataset/labels/query6/dadetv5_query/labels/train/551df69f15ca760f19dd6f40bfda2aa7.txt'),
        '551df69f15ca760f19dd6f40bfda2aa7',
        '/home/chenyi/workspace/yolov5/datasets/dadet100kv4_1_june/labels/train/551df69f15ca760f19dd6f40bfda2aa7.txt',
        'train',
        '/home/chenyi/workspace/dataset/data/data_fashionpedia/train/551df69f15ca760f19dd6f40bfda2aa7.jpg',
        'data_fashionpedia',
        '/home/chenyi/workspace/yolov5/datasets/dadet100kv4_1_june/images/train/551df69f15ca760f19dd6f40bfda2aa7.jpg']],
      dtype=object)

In [546]:
# shbg_df['img_new_sp'] = shbg_df['img_sp'].apply(lambda x: Path(x).parents[1]/Path(x).name)

In [547]:
slink_df.value_counts('source')

source
data_fashionpedia    45529
haowei34k            33399
dada_det             14939
DeepFashion2          4814
shoeBag5k0515         4430
dtype: int64

In [548]:
drop_df = df.drop_duplicates('label_sp')

In [549]:
slink_df = pd.DataFrame()
slink_df['source'] = drop_df['source']
slink_df['fname'] = drop_df['stem']
slink_df['label_sp'] = drop_df['label_sp']
slink_df['img_sp'] = drop_df['img_sp']
slink_df['label_tp'] = drop_df['label_tp']
slink_df['img_tp'] = drop_df['img_tp']

In [41]:
# root = Path('/home/chenyi/workspace/yolov5/datasets/hw_pedia_true/')
# img_root = Path('/home/chenyi/workspace/dataset//')
# yolo_data_root = Path('/home/chenyi/workspace/yolov5/datasets/dataExps/gall10k_delBox/')

In [323]:
# slink_df['label_tp'] = slink_df['label_sp'].apply(lambda x: yolo_data_root /'labels' /x.parts[-2]/x.parts[-1])

In [43]:
slink_df['img_tp'] = slink_df['label_sp'].apply(lambda x: yolo_data_root /'images' /x.parts[-2]/x.parts[-1])
slink_df['img_tp'] = slink_df['img_tp'].apply(lambda x: x.with_suffix('.jpg'))

In [44]:
# slink_df['img_sp'] = slink_df['img_tp'].apply(lambda x: data_root /'images' /x.parts[-2]/x.parts[-1])

In [290]:
slink_df['img_file'] = slink_df['img_sp'].apply(lambda x: Path(x).is_file())
slink_df['label_file'] = slink_df['label_sp'].apply(lambda x: Path(x).is_file())

In [550]:
slink_df.to_csv('/nas/chenyi/workspace/myPyModule/dataset/data/bboxes/dadet100k/dadetv5_query_slink.csv', index=False)

#### 直接读取slink.csv

In [ ]:
# slink_df = pd.read_csv('./data/hw34k_level1_slink.csv')

##### 生成label_tp, img_sp, img_tp

In [551]:
slink_df.tail(2).values

array([['shoeBag5k0515', '2017050718162854629',
        PosixPath('/home/chenyi/workspace/dataset/labels/query6/dadetv5_query/labels/val/2017050718162854629.txt'),
        '/home/chenyi/workspace/dataset/data/dada_det/shoeBag5k0515/images/2017050718162854629.jpg',
        '/home/chenyi/workspace/yolov5/datasets/dadet100kv4_1_june/labels/val/2017050718162854629.txt',
        '/home/chenyi/workspace/yolov5/datasets/dadet100kv4_1_june/images/val/2017050718162854629.jpg'],
       ['shoeBag5k0515', '2018041313545637741',
        PosixPath('/home/chenyi/workspace/dataset/labels/query6/dadetv5_query/labels/val/2018041313545637741.txt'),
        '/home/chenyi/workspace/dataset/data/dada_det/shoeBag5k0515/images/2018041313545637741.jpg',
        '/home/chenyi/workspace/yolov5/datasets/dadet100kv4_1_june/labels/val/2018041313545637741.txt',
        '/home/chenyi/workspace/yolov5/datasets/dadet100kv4_1_june/images/val/2018041313545637741.jpg']],
      dtype=object)

In [552]:
origin_folder = 'dadet100kv4_1_june'
folder_name = 'dadetv5_query'

In [553]:
slink_df['img_tp'] = slink_df['img_tp'].apply(lambda x: str(x).replace(f'/datasets/{origin_folder}/images/', f'/datasets/{folder_name}/images/'))
slink_df['label_tp'] = slink_df['label_tp'].apply(lambda x: str(x).replace(f'/datasets/{origin_folder}/labels/', f'/datasets/{folder_name}/labels/'))

In [554]:
slink_df.to_csv('/nas/chenyi/workspace/myPyModule/dataset/data/bboxes/dadet100k/dadetv5_query_slink.csv', index=False)

### 生成sym link

In [555]:
root = f'/home/chenyi/workspace/yolov5/datasets/'
!tree  -d -L 1 $root

/home/chenyi/workspace/yolov5/datasets/
├── coco128
├── dadet100kv2
├── dadet100kv3_june
├── dadet100kv4_1_june
├── dadet100kv4_june
├── dadet_gallv4_june
├── dadetv5
├── dadetv5_query
├── dataExps
├── deepedia
├── deepedia_catg6
├── deepedia_catg8
├── haowei_bag_shoe
├── hwpedia
├── hwpedia70k
├── hwpedia80k
├── tuneGallery10k
├── tuneGallery5k
├── tuneQuery10k
└── validation_all

20 directories


In [556]:
folder_name = 'dadetv5_query'

In [382]:
# 产生路径
import shutil
# folder_name = 'gall10k_delBox'
folder_name = folder_name
folder_root = Path(root) /folder_name
# folder_root = root
folder1 = ['images', 'labels']
folder2 = ['train', 'val']
# if folder_root.is_dir() and True:
#     shutil.rmtree(folder_root)
for f1 in folder1:
    for f2 in folder2:
        tg = folder_root / f1 / f2
        tg.mkdir(parents=True)  

In [459]:
!tree  -d -L 1 $folder_root

/home/chenyi/workspace/yolov5/datasets/dadetv5_query
├── images
└── labels

2 directories


In [558]:
for p in glob.glob(str(Path(slink_df['img_tp'].values[0]).parents[1]/'*/*.jpg')):
    if Path(p).is_symlink():
        Path(p).unlink()

In [559]:
for p in glob.glob(str(Path(slink_df['label_tp'].values[0]).parents[1]/'*/*.txt')):
    if Path(p).is_symlink():
        Path(p).unlink()

In [560]:
slink_df.head(1)

,source,fname,label_sp,img_sp,label_tp,img_tp
2,data_fashionpedia,551df69f15ca760f19dd6f40bfda2aa7,/home/chenyi/workspace/dataset/labels/query6/d...,/home/chenyi/workspace/dataset/data/data_fashi...,/home/chenyi/workspace/yolov5/datasets/dadetv5...,/home/chenyi/workspace/yolov5/datasets/dadetv5...


In [561]:
%%time
for p in slink_df['img_tp'].values:
    if Path(p).is_symlink():
        Path(p).unlink()
_ = slink_df.apply(lambda row: Path.symlink_to(Path(row['img_tp']), Path(row['img_sp'])), axis=1)

CPU times: user 2.25 s, sys: 1.18 s, total: 3.42 s
Wall time: 3.6 s


In [562]:
%%time
for p in slink_df['label_tp'].values:
    if Path(p).is_symlink():
        Path(p).unlink()
_ = slink_df.apply(lambda row: Path.symlink_to(Path(row['label_tp']), Path(row['label_sp'])), axis=1)

CPU times: user 2.36 s, sys: 1.17 s, total: 3.53 s
Wall time: 4.05 s


In [563]:
len(glob.glob(str(Path(slink_df['img_tp'].values[0]).parents[1]/'*/*.jpg'))), len(glob.glob(str(Path(slink_df['label_tp'].values[0]).parents[1]/'*/*.txt')))

(103109, 103109)

In [564]:
p = str(slink_df['label_tp'].values[1001])
!cat $p

5 0.572266 0.614370 0.367188 0.768328


In [565]:
p = str(slink_df['label_sp'].values[2501])
!cat $p

5 0.573206 0.473633 0.209370 0.460938


#### 核查软连接

In [566]:
import glob,cv2, os
import pandas as pd

In [567]:
!tree  -d -L 1 /home/chenyi/workspace/yolov5/datasets/

/home/chenyi/workspace/yolov5/datasets/
├── coco128
├── dadet100kv2
├── dadet100kv3_june
├── dadet100kv4_1_june
├── dadet100kv4_june
├── dadet_gallv4_june
├── dadetv5
├── dadetv5_query
├── dataExps
├── deepedia
├── deepedia_catg6
├── deepedia_catg8
├── haowei_bag_shoe
├── hwpedia
├── hwpedia70k
├── hwpedia80k
├── tuneGallery10k
├── tuneGallery5k
├── tuneQuery10k
└── validation_all

20 directories


In [568]:
# folder_name = 'dadet100kv2'
root = f'/home/chenyi/workspace/yolov5/datasets/{folder_name}'
!tree  -d -L 2 $root

/home/chenyi/workspace/yolov5/datasets/dadetv5_query
├── images
│   ├── train
│   └── val
└── labels
    ├── train
    └── val

6 directories


In [569]:
def check_syslink(root, mode='images'):
    subfix = '.txt' if mode=='labels' else '.jpg'
    flag_list = [os.path.isfile(os.readlink(p)) for p in glob.glob(f'{root}/{mode}/*/*{subfix}')]
    txt_df = pd.DataFrame()
    txt_df['flag'] = flag_list
    return txt_df.value_counts('flag')

In [570]:
%%time
check_syslink(root, mode='images')

CPU times: user 201 ms, sys: 236 ms, total: 437 ms
Wall time: 495 ms


flag
True    103109
dtype: int64

In [571]:
%%time
check_syslink(root, mode='labels')

CPU times: user 190 ms, sys: 193 ms, total: 383 ms
Wall time: 381 ms


flag
True    103109
dtype: int64